In [ ]:
#!pip istall coloroma

In [ ]:
import random
import copy
from colorama import Fore

# Draughts Endgame : 4 Kings x 2 Kings

In the end I decided to create a game of drafts following an endgame based on the following:

https://lidraughts.org/study/YiAZbWM6

I decided to go with 4 kings vs 2 kings.

In this scenario, there will be only one 'type' of move. Kings are able to move diagonally forward and backwards, as well as capture other pieces.

## Draughts Class
Class which determines start position of pieces, board state, and display the board

In [ ]:
class CurrentBoard:

    BOARD_SIZE = 8
    PLAYER_COUNT = 6

    def __init__(self, board=None):
        if board is None:
            self.board = [[" " for _ in range(self.BOARD_SIZE)] for _ in range(self.BOARD_SIZE)]
        else:
            self.board = board
        self.player_positions = {"W": [], "B": []}
        self.state = "U"
        

    def initialise_board(self):
        start_positons = random.sample(range(self.BOARD_SIZE*self.BOARD_SIZE), self.PLAYER_COUNT)

        white_kings = start_positons[:2]
        black_kings = start_positons[2:]

        for index in white_kings:
            row = index // self.BOARD_SIZE
            col = index % self.BOARD_SIZE
            self.board[row][col] = "W"
            self.player_positions["W"].append((row,col))
        
        for index in black_kings:
            row = index // self.BOARD_SIZE
            col = index % self.BOARD_SIZE
            self.board[row][col] = "B"
            self.player_positions["B"].append((row,col))

        self.state = self.state_of_board()

    def other(self,piece):
        if piece == "B":
            return "W"
        return "B"


    def display_board(self, game_display=False):
        if game_display:
            indx = 0

            print(Fore.BLUE + "+" + "----+" * self.BOARD_SIZE)
            for row in range(self.BOARD_SIZE):
                print("|", end="")
                for col in range(self.BOARD_SIZE):
                    if self.board[row][col] == "B" or self.board[row][col] == "W":
                        print(" " + str(Fore.WHITE + self.board[row][col]) + Fore.BLUE + "  |", end="")
                    else:
                        if indx < 10:
                            print(" " + str(indx) + "  |", end="")
                        else:
                            print(" " + str(indx) + " |", end="")
                    indx += 1
                print()
                print("+" + "----+" * self.BOARD_SIZE)
        else:
            print("+" + "----+" * self.BOARD_SIZE)
            for row in range(self.BOARD_SIZE):
                print("|", end="")
                for col in range(self.BOARD_SIZE):
                    print(" " + str(self.board[row][col]) + "  |", end="")
                print()
                print("+" + "----+" * self.BOARD_SIZE)

    def state_of_board(self):
        
        if len(self.all_possible_moves("W")) == 0 or len(self.all_possible_moves("B")) == 0:
            return "W"
        elif len(self.all_possible_moves("W")) > 0 or len(self.all_possible_moves("B")) > 0:
            return "U"
        else:
            return "D"

    def move_piece_on_board(self, piece, curr_index, destination_index):

        # need to check if move is in possible moves but had issues and ran out of time

        # possible_moves = self.all_possible_moves(piece)
        # loop through moves and check against
            
        curr_row = curr_index // self.BOARD_SIZE
        curr_col = curr_index % self.BOARD_SIZE

        new_row = destination_index // self.BOARD_SIZE
        new_col = destination_index % self.BOARD_SIZE

        capture_row = curr_row
        capture_col = curr_col

        while capture_row != new_row or capture_col != new_col:

            if new_row > curr_row:
                capture_row += 1
            elif new_row < curr_row:
                capture_row -= 1

            if new_col > curr_col:
                capture_col += 1
            elif new_col < curr_col:
                capture_col -= 1

            if self.board[capture_row][capture_col] != " " and self.board[capture_row][capture_col] != piece:
                # Capture the opponent's piece
                self.board[capture_row][capture_col] = " "
                print("Piece captured!")

        self.board[curr_row][curr_col] = " "
        self.board[new_row][new_col] = piece


        self.state = self.state_of_board()
        #self.display_board()
        # else:
        #     print("Invalid move")
        return self



    # In this case the only moves are king moves. They can move diagonally in any direction as many spaces
    def all_possible_moves(self, player_piece):
        possible_moves = []

        player_positions = self.player_positions[player_piece]
        move_directions = [(1, -1), (1, 1), (-1, -1), (-1, 1)] # f-l, f-r, b-l, b-r

        for row, col in player_positions:
            for rowd, cold in move_directions:

                new_row, new_col = row + rowd, col + cold

                for _ in range(self.BOARD_SIZE - 1):
                    if 0 <= new_row < self.BOARD_SIZE and 0 <= new_col < self.BOARD_SIZE:
                        if self.board[new_row][new_col] == " ":

                            new_board = copy.deepcopy(self.board)
                            new_player_positions = copy.deepcopy(self.player_positions)
                            new_board[row][col] = " "
                            new_board[new_row][new_col] = player_piece
                            new_player_positions[player_piece] = [(new_row, new_col)]

                            possible_moves.append(CurrentBoard(new_board))
                        else:
                            break
                        new_row += rowd
                        new_col += cold
                    else:
                        break

        return possible_moves
        


#### Test CurrentBoard Class

In [ ]:
cb = CurrentBoard()
cb.initialise_board()
cb.display_board(True)

In [ ]:
cb.all_possible_moves("W")

In [ ]:
cb.all_possible_moves("W")[0].display_board()

In [ ]:
cb.player_positions

In [ ]:
cb.state_of_board()

### Search Tree Node class

In [ ]:
class SearchTreeNode:

    def __init__(self,board_instance,playing_as, ply=0):

        self.children = []
        self.max_ply_depth = 3
        self.value_is_assigned = False
        self.ply_depth = ply
        self.current_board = board_instance
        self.move_for = playing_as
        
        if self.current_board.state == "U":
            if self.ply_depth <= self.max_ply_depth:
                self.generate_children()
                # evaluation function code based on pieces left and control of center rxc : extract function
                player_pieces_left_count = sum(row.count(playing_as) for row in self.current_board.board)
                center_control_score = self.count_center_pieces(playing_as)
                self.value = player_pieces_left_count + center_control_score
                self.value_is_assigned = True
            else:
                # evaluation function code based on pieces left and control of center rxc
                player_pieces_left_count = sum(row.count(playing_as) for row in self.current_board.board)
                center_control_score = self.count_center_pieces(playing_as)
                self.value = player_pieces_left_count + center_control_score
                self.value_is_assigned = True
        else:  # Game over
            if self.current_board.state == "D":
                self.value = 0
            else:
                if ((self.ply_depth % 2) == 0):
                    self.value = -1000
                else:
                    self.value = 1000
            self.value_is_assigned = True

    # check center 4 squares
    def count_center_pieces(self, player):
        center_row_start = self.current_board.BOARD_SIZE // 2 - 2
        center_row_end = center_row_start + 3
        center_col_start = self.current_board.BOARD_SIZE // 2 - 2
        center_col_end = center_col_start + 3
        
        center_count = 0
        for row in range(center_row_start, center_row_end):
            for col in range(center_col_start, center_col_end):
                if self.current_board.board[row][col] == player:
                    center_count += 1
        return center_count

    def min_max_value(self):
        if self.value_is_assigned:
            return self.value

        self.children  = sorted(self.children, key = lambda x:x.min_max_value())

        if ((self.ply_depth % 2) == 0):
            self.value = self.children[-1].value
        else:
            self.value = self.children[0].value
        self.value_is_assigned = True

        return self.value

    def generate_children(self):
        for board_for_next_move in self.current_board.all_possible_moves(self.move_for):
            self.children.append(SearchTreeNode(board_for_next_move, self.current_board.other(self.move_for), ply=self.ply_depth + 1))


### Test SearchTreeNode Class

In [ ]:
cb = CurrentBoard()
cb.initialise_board()

In [ ]:
stn = SearchTreeNode(cb, "W")

In [ ]:
stn.children

In [ ]:
stn.children[0].current_board.display_board()

In [ ]:
cb.display_board()

In [ ]:
stn.count_center_pieces("B")

In [ ]:
stn.min_max_value()

## PLay Draughts Class

In [ ]:
def play_draughts():

    MAX_MOVES = 10

    response = input("Do you want to play first? (y/n)")
    players_turn = (response == "y")

    response = input("Do you want to play as black or white? (b/w)")
    

    if response == "b":
        player_is_playing = "B"
    else:
        player_is_playing = "W"

    cb = CurrentBoard()
    cb.initialise_board()

    for _ in range(MAX_MOVES):

        if players_turn:
            cb.display_board(game_display=True)

            piece_choice = None

            while piece_choice is None:
                try:
                    piece_choice = int(input("Select the index of the piece to move: "))
                    if piece_choice < 0 or piece_choice >= 64:
                        print("Invalid, please try again (0 and 63)")
                    elif cb.board[piece_choice // 8][piece_choice % 8] != player_is_playing:
                        print("Invalid, the selected piece does not belong to you")
                        piece_choice = None
                except ValueError:
                    print("Invalid input, please enter a number")

            destination_choice = None

            while destination_choice is None:
                try:
                    destination_choice = int(input("Select the index of the destination square: "))
                    if destination_choice < 0 or destination_choice >= 64:
                        print("Invalid, please try again (0 and 63)")
                    elif cb.board[destination_choice // 8][destination_choice % 8] != ' ':
                        print("Invalid, the selected position is occupied")
                        destination_choice = None
                except ValueError:
                    print("Invalid input, please enter a number")

            cb = cb.move_piece_on_board(player_is_playing, piece_choice, destination_choice)
            cb.display_board()
            
        # not players turn    
        else:
            search_tree = SearchTreeNode(cb, cb.other(player_is_playing))
            print("player two", cb.other(player_is_playing))
            search_tree.min_max_value()
            cb = search_tree.children[-1].current_board
            print("Other player")
            cb.display_board()

        if cb.state != "U":
            if cb.state == "D":
                print("It's a draw")
            else:
                if players_turn:
                    print("You win")
                else:
                    print("You lose!")

                cb.display_board()
            break

        players_turn = not players_turn


## Play the Game

In [ ]:
play_draughts()

There is an issue with playing the game but I am pulling my hair out. Issues in game logic as well but I tried my best!